<a href="https://colab.research.google.com/github/Lailson001/E-comerce-modelo/blob/main/An%C3%A1lise%20E-comerce.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

import numpy as np
import matplotlib




# Nova seção

No primeiro passo da análise, realizamos uma inspeção geral dos dados utilizando df.info(). Identificamos que o DataFrame contém 12.575 registros e 11 colunas, com diferentes tipos de dados e valores ausentes em algumas colunas, como Item e Discount Applied

In [27]:
df= pd.read_csv("/retail_store_sales.csv")
df.head()
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12575 entries, 0 to 12574
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    12575 non-null  object 
 1   Customer ID       12575 non-null  object 
 2   Category          12575 non-null  object 
 3   Item              11362 non-null  object 
 4   Price Per Unit    11966 non-null  float64
 5   Quantity          11971 non-null  float64
 6   Total Spent       11971 non-null  float64
 7   Payment Method    12575 non-null  object 
 8   Location          12575 non-null  object 
 9   Transaction Date  12575 non-null  object 
 10  Discount Applied  8376 non-null   object 
dtypes: float64(3), object(8)
memory usage: 1.1+ MB
None


Ao indentificar os valores ausentes, usamos cálculos para preenchê-los com dados coerentes. Por exemplo, completamos Total Spent multiplicando Quantity por Price Per Unit. Isso ajuda a manter a consistência das informações.

In [28]:
df['Total Spent'] = df['Total Spent'].fillna(df['Quantity']* df['Price Per Unit'])
df['Price Per Unit'] = df[ 'Price Per Unit'].fillna(df['Total Spent'] / df['Quantity'])
print(df.isnull().sum())

Transaction ID         0
Customer ID            0
Category               0
Item                1213
Price Per Unit         0
Quantity             604
Total Spent          604
Payment Method         0
Location               0
Transaction Date       0
Discount Applied    4199
dtype: int64


In [38]:
# Verificando inconsistências com tolerância de erro
df['Calculated Total'] = df['Quantity'] * df['Price Per Unit']

inconsistentes = df[~np.isclose(df['Total Spent'], df['Calculated Total'], atol=0.01)]

print(f"Existem {len(inconsistentes)} transações inconsistentes.")


Existem 604 transações inconsistentes.
